<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 09:16:19
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 22.57 K (0.17%)
Current PnL: -21.61 L (-14.7%)
CY Booked + Current PnL: -13.34 L (-9.07%)
-------------------
Total profit:  2.64 L
Total loss:  -24.25 L
-------------------
Total Booked + Current PnL: 13.64 L (11.37%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.08 L (60.84%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.26,112048.0,7736.0,8919.0,-0.03,7.42,7.96,15.96,11.0,0.87,0.86,25.57,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,5.95,172623.0,16446.0,9650.0,0.49,10.53,5.59,16.71,19.0,1.70,1.32,29.64,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-12.30,M-LC,3.80,217724.0,16244.0,10973.0,-0.03,8.06,5.04,13.51,61.0,1.48,1.67,21.68,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,10.21,H-MC,13.76,157366.0,31166.0,15013.0,0.07,24.70,9.54,36.59,111.0,2.08,1.20,40.30,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-24.44,H-SC,10.21,225685.0,32665.0,15369.0,0.71,16.92,6.81,24.89,134.0,2.13,1.73,44.22,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.99,H-SC,4.63,101590.0,-910.0,41743.0,0.58,-0.89,41.09,39.83,116.0,-0.02,0.78,0.58,AR,ATH,MISC
25,DABUR,505.20,735.00,-9.78,H-MC,3.49,196482.0,-546.0,90166.0,0.06,-0.28,45.89,45.49,103.0,-0.01,1.50,12.73,XY24,BTT,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100668.0,164.0,30472.0,1.83,0.16,30.27,30.49,80.0,0.01,0.77,32.62,MH,ATH,METALS
92,WIPRO,243.46,420.00,-14.25,M-LC,7.39,150939.0,-6.0,109461.0,0.74,-0.00,72.52,72.51,54.0,-0.00,1.15,6.02,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.09,H-LC,2.56,215528.0,11028.0,30389.0,0.78,5.39,14.10,20.25,10.0,0.36,1.65,14.60,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.54,H-LC,3.16,170551.0,-30835.0,118448.0,0.32,-15.31,69.45,43.51,27.0,-0.26,1.30,18.72,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.37,H-LC,3.53,128583.0,-136843.0,193749.0,0.19,-51.56,150.68,21.44,8.0,-0.71,0.98,3.99,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-6.65,H-LC,3.74,184950.0,-15360.0,75330.0,0.45,-7.67,40.73,29.94,17.0,-0.20,1.41,27.77,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-11.16,H-LC,5.80,279451.0,9104.0,25514.0,-0.00,3.37,9.13,12.81,28.0,0.36,2.14,16.87,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.00,-13.84,H-MC,5.40,181745.0,-18761.0,67536.0,2.08,-9.36,37.16,24.33,98.0,-0.28,1.39,4.80,X40N,BTT,FINANCE
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100668.0,164.0,30472.0,1.83,0.16,30.27,30.49,80.0,0.01,0.77,32.62,MH,ATH,METALS
26,DEN,52.79,75.00,104.66,M-SC,1.65,57264.0,-27200.0,62738.0,1.45,-32.20,109.56,42.07,236.0,-0.43,0.44,21.61,AR,NTT,ENTERTAINMENT
91,WHIRLPOOL,1219.98,2270.00,-44.20,M-SC,4.22,94208.0,2710.0,76045.0,1.19,2.96,80.72,86.07,219.0,0.04,0.72,36.28,XR,NTT,DURABLES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMPUS,294.86,393.00,-30.83,M-SC,2.17,143985.0,-23790.0,79638.0,-2.93,-14.18,55.31,33.28,208.0,-0.30,1.10,11.48,XY24,NTT,FOOTWEAR
70,SHALBY,261.39,327.00,938.28,M-SC,6.89,141123.0,-40020.0,85492.0,-2.27,-22.09,60.58,25.10,232.0,-0.47,1.08,13.10,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.53,72230.0,-41319.0,81367.0,-1.26,-36.39,112.65,35.27,268.0,-0.51,0.55,76.79,XR,NTT,HOTELS
44,INDUSINDBK,1354.50,1800.00,259.24,L-MC,8.47,44087.0,-33120.0,58512.0,-1.23,-42.90,132.72,32.89,258.0,-0.57,0.34,21.41,XR,NTT,BANKS
90,VOLTAS,1278.28,1918.49,-9.61,H-MC,7.75,189525.0,-2217.0,98250.0,-0.99,-1.16,51.84,50.08,101.0,-0.02,1.45,5.56,XY25,ATH,AC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.38,M-SC,10.33,83355.0,-17422.0,17513.0,0.20,-17.29,21.01,0.09,243.0,-0.99,0.64,9.51,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,82.03,H-SC,20.44,114858.0,-27621.0,47138.0,-0.42,-19.39,41.04,13.70,163.0,-0.59,0.88,35.56,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.09,H-SC,3.12,115398.0,-28405.0,55149.0,0.01,-19.75,47.79,18.60,148.0,-0.52,0.88,27.35,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-69.07,H-SC,2.79,213219.0,-56448.0,92772.0,0.18,-20.93,43.51,13.47,142.0,-0.61,1.63,7.84,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,107.03,M-SC,2.93,135644.0,-38915.0,38943.0,-0.48,-22.29,28.71,0.02,221.0,-1.00,1.04,17.62,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.04,H-LC,15.95,237432.0,-58502.0,152408.0,0.26,-19.77,64.19,31.73,2.0,-0.38,1.82,1.37,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,10.94,258333.0,-3268.0,148903.0,1.16,-1.25,57.64,55.67,4.0,-0.02,1.98,4.78,X40,BTT,IT
89,VBL,492.64,671.64,-9.80,H-LC,10.11,273734.0,8694.0,87595.0,-0.45,3.28,32.00,36.33,5.0,0.10,2.09,17.02,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.52,H-LC,10.43,198412.0,-18107.0,111012.0,0.01,-8.36,55.95,42.91,7.0,-0.16,1.52,3.91,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.37,H-LC,3.53,128583.0,-136843.0,193749.0,0.19,-51.56,150.68,21.44,8.0,-0.71,0.98,3.99,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.21,H-MC,13.76,157366.0,31166.0,15013.0,0.07,24.7,9.54,36.59,111.0,2.08,1.2,40.3,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,10.21,H-MC,13.76,157366.0,31166.0,15013.0,0.07,24.70,9.54,36.59,111.0,2.08,1.20,40.30,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,8.64,H-SC,12.02,129528.0,4231.0,54777.0,-0.28,3.38,42.29,47.09,141.0,0.08,0.99,62.11,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.52,H-MC,6.97,175240.0,10570.0,57391.0,-0.19,6.42,32.75,41.28,91.0,0.18,1.34,14.91,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.16,H-SC,3.89,107870.0,5866.0,38057.0,0.81,5.75,35.28,43.06,123.0,0.15,0.82,23.85,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-10.60,H-MC,7.85,166166.0,15253.0,42057.0,-0.44,10.11,25.31,37.97,92.0,0.36,1.27,37.97,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-10.60,H-MC,7.85,166166.0,15253.0,42057.0,-0.44,10.11,25.31,37.97,92.0,0.36,1.27,37.97,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,11.99,135155.0,11819.0,121937.0,-0.11,9.58,90.22,108.45,119.0,0.10,1.03,34.08,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,8.64,H-SC,12.02,129528.0,4231.0,54777.0,-0.28,3.38,42.29,47.09,141.0,0.08,0.99,62.11,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.20,M-SC,4.22,94208.0,2710.0,76045.0,1.19,2.96,80.72,86.07,219.0,0.04,0.72,36.28,XR,NTT,DURABLES
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100668.0,164.0,30472.0,1.83,0.16,30.27,30.49,80.0,0.01,0.77,32.62,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.53,72230.0,-41319.0,81367.0,-1.26,-36.39,112.65,35.27,268.0,-0.51,0.55,76.79,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.16,H-SC,15.64,90089.0,-10874.0,105404.0,0.10,-10.77,117.00,93.63,149.0,-0.10,0.69,34.19,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.15,H-SC,13.90,92115.0,-5865.0,27736.0,-0.23,-5.99,30.11,22.32,152.0,-0.21,0.70,33.60,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.26,112048.0,7736.0,8919.0,-0.03,7.42,7.96,15.96,11.0,0.87,0.86,25.57,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-24.44,H-SC,10.21,225685.0,32665.0,15369.0,0.71,16.92,6.81,24.89,134.0,2.13,1.73,44.22,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-10.60,H-MC,7.85,166166.0,15253.0,42057.0,-0.44,10.11,25.31,37.97,92.0,0.36,1.27,37.97,AR,ATH,AUTO
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.53,72230.0,-41319.0,81367.0,-1.26,-36.39,112.65,35.27,268.0,-0.51,0.55,76.79,XR,NTT,HOTELS
70,SHALBY,261.39,327.00,938.28,M-SC,6.89,141123.0,-40020.0,85492.0,-2.27,-22.09,60.58,25.10,232.0,-0.47,1.08,13.10,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-9.80,H-LC,10.11,273734.0,8694.0,87595.0,-0.45,3.28,32.00,36.33,5.0,0.10,2.09,17.02,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,41.19
2,50,70.89


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.41
LC    34.36
MC    26.22
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.86
X40N     15.54
X40      13.33
XY25     10.65
XR        9.06
AR        8.44
OX40N     7.51
X5K       2.29
MH        1.76
X200      1.41
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.99
H-SC    24.61
H-MC    23.25
M-SC    13.32
M-LC     7.28
M-MC     2.63
L-SC     1.48
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.22,-10.84,46.47
IT,11.99,-19.08,78.56
FINANCE,8.27,-20.43,64.99
BANKS,7.64,-14.21,62.70
MISC,5.90,-19.11,87.39
PAINTS,5.62,-18.73,42.12
HEALTHCARE,5.33,5.78,18.01
ELECTRICAL,4.71,-14.05,63.44
AUTO,4.49,-15.04,65.09


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2862548.0,23
XR,1107237.0,13
AR,1105729.0,10
X40,829715.0,10
XY25,655952.0,8
X40N,575396.0,12
OX40N,476778.0,10
SR,194970.0,2
X5K,139295.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2798778.0,28
M-SC,1503528.0,17
H-LC,1338834.0,18
H-MC,1336630.0,18
M-LC,426613.0,5
M-MC,334091.0,2
L-SC,267096.0,3
L-MC,58512.0,1
L-LC,44117.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      939792.0      6
M-SC       XY24      877746.0      7
H-SC       AR        672199.0      5
           XR        519931.0      6
H-LC       X40       515223.0      5
H-MC       XY24      495855.0      5
M-MC       XY24      334091.0      2
H-LC       XY25      254613.0      3
           AR        231557.0      2
H-MC       X40       231338.0      4
           XY25      222920.0      2
H-LC       X40N      213931.0      5
H-SC       X40N      213228.0      4
           OX40N     203881.0      4
           SR        194970.0      2
L-SC       XR        177153.0      2
M-LC       XY24      175803.0      2
M-SC       AR        159916.0      2
H-MC       X40N      148237.0      3
M-SC       OX40N     134361.0      4
M-LC       X5K       130376.0      1
M-SC       XR        125022.0      2
           XY25      123329.0      1
H-MC       XR        117158.0      1
M-LC       XR        109461.0      1
L-SC       OX40N      89943.0      1
M-SC       X40        83154.0      1
H-LC       X200       75330.0      1
L-MC       XR         58512.0      1
H-SC       MH         54777.0      1
H-MC       OX40N      48593.0      1
L-LC       XY25       44117.0      1
H-MC       AR         42057.0      1
H-LC       XY24       39261.0      1
H-MC       MH         30472.0      1
M-LC       XY25       10973.0      1
H-LC       X5K         8919.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
